In [1]:
import csv
import pandas as pd

import flywheel
fw = flywheel.Client('', root=True)  # This works if you are logged into a Flywheel instance on a Terminal.
fw.get_config().site.api_url  # Prints the instance you are logged into to make sure it is the right one.

/usr/local/lib/python3.9/site-packages/flywheel/flywheel.py:6274: UserWarning: Client version 15.3.1 does not match server version 14.6.2. Please update your client version!
  warnings.warn('Client version {} does not match server version {}. Please update your client version!'.format(SDK_VERSION, release_version))


'https://rollout.ce.flywheel.io/api'

In [13]:
def save_curation_csvs(group_label, project_label):
    """Save BIDS mapping from acquisition names to BIDS paths and fieldmap IntendedFors as sorted csv files."""

    project = fw.projects.find_one(f'group={group_label},label={project_label}')

    intended_for_acquisition = dict()
    intended_for_dirs = dict()
    intended_fors = dict()
    
    nifti_df = pd.DataFrame(columns=("Acquisition label (SeriesDescription)", "File name", "File type", "Curated BIDS path"))

    ii = 0  # Current acquisition index
    for acquisition in fw.acquisitions.iter_find(f"project={project.id}"):

        print(ii, "  ", acquisition.label)
        for file in acquisition.reload().files:
            if file.info['BIDS'] == "NA":
                print(f"{acquisition.label}, {file.name}, {file.type}, nonBids")
                nifti_df.loc[ii] = [acquisition.label, 
                                    file.name,
                                    file.type,
                                    f"nonBids"]
            else:
                if not file.info['BIDS']['ignore']:
                    print(f"{acquisition.label}, {file.name}, {file.type}, {file.info['BIDS']['Folder']}/{file.info['BIDS']['Filename']}")
                    nifti_df.loc[ii] = [acquisition.label, 
                                        file.name,
                                        file.type,
                                        f"{file.info['BIDS']['Folder']}/{file.info['BIDS']['Filename']}"]
                else:
                    print(f"{acquisition.label}, {file.name}, {file.type}, Ignored")
                    nifti_df.loc[ii] = [acquisition.label, file.name, file.type, "Ignored"]
            if 'IntendedFor' in file.info['BIDS']:
                intended_for_acquisition[file.name] = acquisition.label
                intended_for_dirs[file.name] = file.info['BIDS']['IntendedFor']
                intended_fors[file.name] = file.info['IntendedFor']
            ii += 1
    
    nifti_df.sort_values(by=["Curated BIDS path"], inplace=True)
    
    nifti_df.to_csv(f'{group_label}_{project_label}_niftis.csv', index=False)  
           
    print()

    with open(f'{group_label}_{project_label}_intendedfors.csv', mode='w') as intendedfors_file:
        intendedfors_writer = csv.writer(intendedfors_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        intendedfors_writer.writerow(["acquisition label", "file name and folder", "IntendedFor List of BIDS paths"])

        for k,v in intended_for_dirs.items():
            print(f"{intended_for_acquisition[k]}, {k}")
            intendedfors_writer.writerow([intended_for_acquisition[k], k])
            for i in v:
                print(f",{i['Folder']}")
                intendedfors_writer.writerow(["",i['Folder']])
                intended_fors[k].sort()
                for j in intended_fors[k]:
                    print(f",,{j}")
                    intendedfors_writer.writerow(["","",j])

    return nifti_df

In [14]:
df = save_curation_csvs('bids-curation-tests','dcm_qa_nih-master')

0    4 - Axial EPI-FMRI (Interleaved I to S)
4 - Axial EPI-FMRI (Interleaved I to S), 4 - Axial EPI-FMRI (Interleaved I to S).dicom.zip, dicom, sourcedata/4 - Axial EPI-FMRI (Interleaved I to S).dicom.zip
4 - Axial EPI-FMRI (Interleaved I to S), 4_Axial_EPI_FMRI_Interleaved_I_to_S.nii.gz, nifti, func/sub-DEV_ses-20180918114023_task-{file.info.BIDS.Task}_bold.nii.gz
2    5 - Axial EPI-FMRI (Sequential I to S)
5 - Axial EPI-FMRI (Sequential I to S), 5 - Axial EPI-FMRI (Sequential I to S).dicom.zip, dicom, sourcedata/5 - Axial EPI-FMRI (Sequential I to S).dicom.zip
5 - Axial EPI-FMRI (Sequential I to S), 5_Axial_EPI_FMRI_Sequential_I_to_S.nii.gz, nifti, func/sub-DEV_ses-20180918114023_task-{file.info.BIDS.Task}_bold.nii.gz
4    6 - Axial EPI-FMRI (Interleaved S to I)
6 - Axial EPI-FMRI (Interleaved S to I), 6 - Axial EPI-FMRI (Interleaved S to I).dicom.zip, dicom, sourcedata/6 - Axial EPI-FMRI (Interleaved S to I).dicom.zip
6 - Axial EPI-FMRI (Interleaved S to I), 6_Axial_EPI_FMRI_Interle

In [15]:
df

,Acquisition label (SeriesDescription),File name,File type,Curated BIDS path
1,4 - Axial EPI-FMRI (Interleaved I to S),4_Axial_EPI_FMRI_Interleaved_I_to_S.nii.gz,nifti,func/sub-DEV_ses-20180918114023_task-{file.inf...
3,5 - Axial EPI-FMRI (Sequential I to S),5_Axial_EPI_FMRI_Sequential_I_to_S.nii.gz,nifti,func/sub-DEV_ses-20180918114023_task-{file.inf...
5,6 - Axial EPI-FMRI (Interleaved S to I),6_Axial_EPI_FMRI_Interleaved_S_to_I.nii.gz,nifti,func/sub-DEV_ses-20180918114023_task-{file.inf...
7,7 - Axial EPI-FMRI (Sequential S to I),7_Axial_EPI_FMRI_Sequential_S_to_I.nii.gz,nifti,func/sub-DEV_ses-20180918114023_task-{file.inf...
8,3 - EPI PE=AP,3 - EPI PE=AP.dicom.zip,dicom,nonBids
9,3 - EPI PE=AP,3_EPI_PE_AP.nii.gz,nifti,nonBids
10,4 - EPI PE=PA,4 - EPI PE=PA.dicom.zip,dicom,nonBids
11,4 - EPI PE=PA,4_EPI_PE_PA.nii.gz,nifti,nonBids
12,5 - EPI PE=RL,5 - EPI PE=RL.dicom.zip,dicom,nonBids
13,5 - EPI PE=RL,5_EPI_PE_RL.nii.gz,nifti,nonBids
